In [1]:
data = [
    ['E', 'M', 'B'],
    ['E', 'M'],
    ['B', 'W', 'Z'],
    ['W', 'Z'],
    ['E', 'B', 'W'],
    ['W', 'Z'],
    ['M', 'W', 'Z']
    ]

# Apriori

In [22]:
#TODO nicht korrekt
import itertools
class Apriori:
    def __init__(self, data, threshold=1):
        self._data=data
        self._threshold=threshold
        self._keys = self._get_keys()
                
    def _get_keys(self):
        keys=[]
        for d in self._data:
            for j in d:
                if not (j in keys):
                    keys.append(j)
        return keys
    
    def thresholded(self,all_sets):
    
        thresholded_sets={}
        for key, value in all_sets.items():
            if value >= self._threshold:
                thresholded_sets[key]=value
        return thresholded_sets       
    
    def __call__(self):
        n=1
        while True:
            l_n =self._get_l_n(n)
            if not l_n: break
            print(f"L_{n}: {l_n}")
            n+=1
    
    
    def _get_l_n(self, gen):
        all_sets={}
        for values in itertools.combinations(list(self._keys), gen):
            for d in self._data: 
                if all(v in d for v in values):
                    all_sets[values]=all_sets.get(values,0)+1
        print(f"C_{gen}:{all_sets}")
        return self.thresholded(all_sets)     
    
            
        
apriori=Apriori(data,2)
apriori()

C_1:{('b',): 3, ('e',): 3, ('m',): 2, ('w',): 3, ('z',): 2}
L_1: {('b',): 3, ('e',): 3, ('m',): 2, ('w',): 3, ('z',): 2}
C_2:{('b', 'e'): 1, ('b', 'w'): 1, ('e', 'm'): 1, ('w', 'z'): 1}


n: 1, {'E': 3, 'M': 3, 'B': 3, 'W': 5, 'Z': 4}


# Lift

In [2]:
transactions=7
data={
    "b":3,
    "e":3,
    "m":3,
    "w":5,
    "z":4,
    "be":2,
    "bw":2,
    "em":2,
    "wz":4
    }


In [20]:
def calc_lift(P_a,P_b,P_ab):
    lift = P_ab/(P_a*P_b)
    output=f"lift(A=>B)=({round(P_ab,3)} / ({round(P_a,3)} * {round(P_b,3)}) = {round(lift,3)}"
    if lift<1:
        result="negativ korreliert"
    elif lift==1:
        result="unkorreliert"
    else:
        result="positiv korreliert"
    return output + f" -> {result}"

class P:
    def __init__(self, transactions):
        self.transactions=transactions
    def __call__(self,A):
        return A/transactions

def lift(transactions,A,B,AB):
    p=P(transactions)
    return calc_lift(p(A),p(B),p(AB)).replace("A","w").replace("B","z")

In [21]:
d=data
t=transactions

lift(t,d["w"],d["z"],d["wz"])

'lift(w=>z)=(0.571 / (0.714 * 0.571) = 1.4 -> positiv korreliert'

# Null-invariante

In [32]:
P_milch=1300/3000
P_eier=1200/3000
P_miei=700/3000

## All-Confidence-Maß

In [34]:
def all_conf(P_a,P_b,P_ab):
    return P_ab/max(P_a,P_b)

round(all_conf(P_milch,P_eier,P_miei),3)

0.538

## Max-Confidence-Maß

In [35]:
def max_conf(P_a,P_b,P_ab):
    return P_ab/min(P_a,P_b)

round(max_conf(P_milch,P_eier,P_miei),3)

0.583

## Kulczynski-Maß

In [36]:
def kulc(P_a,P_b,P_ab):
    return (1/2)*(P_ab/P_a + P_ab/P_b)

round(kulc(P_milch,P_eier,P_miei),3)

0.561

## Cosine Similarity

In [37]:
from math import sqrt

def cosine(P_a,P_b,P_ab):
    return sqrt(P_ab/P_a * P_ab/P_b)

round(cosine(P_milch,P_eier,P_miei),3)

0.56

## jaccard coefficient

In [38]:
def jacc(P_a,P_b,P_ab):
    return P_ab/(P_a + P_b - P_ab)

round(jacc(P_milch,P_eier,P_miei),3)

0.389

## All together

In [42]:
class P:
    def __init__(self, transactions):
        self.transactions=transactions
    def __call__(self,A):
        return A/transactions
p=P(transactions)
P_A=p(d["w"])
P_B=p(d["z"])
P_AB=p(d["wz"])

functions=[all_conf,max_conf,kulc,cosine,jacc]
for func in functions:
    print(f"{func.__name__}: {round(func(P_A,P_B,P_AB),4)}")

all_conf: 0.8
max_conf: 1.0
kulc: 0.9
cosine: 0.8944
jacc: 0.8
